# Mise en forme des dossiers de travail

In [1]:
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [2]:
import os
if os.getcwd().split('\\')[-1]!='OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation':
    # si valeur diff 'C:\\Users\\...\\Projets\\P6\\OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation'
    os.chdir('OC-DS-P6-Classifiez-automatiquement-des-biens-de-consommation')

In [5]:
import tensorflow_core as tfc
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))#tf.config.list_physical_devices('GPU')
print('la version de tensorflow est: ', tf.__version__)
print('la version de tensorflow_core est: ', tfc.__version__)

In [2]:
IMAGE_SIZE = (224,224)
train_path = 'data/cleaned/Images/train'
test_path  = 'data/cleaned/Images/test'
all_path   = 'data/cleaned/Images/all'

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

if not os.path.exists( train_path ): 
    os.makedirs( train_path )
    
if not os.path.exists( test_path ): 
    os.makedirs( test_path )

for repertoire in df.cat_1.unique():
    
    Xdf = df.loc[df.cat_1==repertoire]
    #print(df.iloc[i][['cat_1']].values[0])
    
    # Create the repositorys only if it doesn't existe yet
    for t in [train_path, test_path,all_path]:

        if not os.path.exists( t+"_".join(repertoire.split(' ')) ): 
            os.makedirs( t+"_".join(repertoire.split(' ')) )
    
    # create train test split
    X_train, X_test, y_train, y_test = train_test_split( Xdf[['image']], Xdf[['cat_1']], test_size=0.33, random_state=42)
    
    # resize and write .jpg train files
    for img in X_train.values:  
        image = Image.open('data/source/raw_folder_image/'+img[0]) 

        # Resizing image
        new_image = image.resize((224, 224))

        # Sauvegarde image format JPEG
        new_image.save(train_path+'/'+"_".join(repertoire.split(' '))+'/'+img[0],format = 'JPEG')
    
    # resize and write .jpg test files
    for img in X_test.values:  
        image = Image.open('data/source/raw_folder_image/'+img[0]) 

        # Resizing image
        new_image = image.resize((224, 224))

        # Sauvegarde image format JPEG
        new_image.save(test_path+'/'+"_".join(repertoire.split(' '))+'/'+img[0],format = 'JPEG')

        

ImportError: DLL load failed: Le module spécifié est introuvable.

## Utilisation du VGG-16 pré-entraîné

In [1]:
from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

Using TensorFlow backend.


553467904/553467096 [==============================] - 32s 0us/step


In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img('cat.jpg', target_size=(224, 224))  # Charger l'image
img = img_to_array(img)  # Convertir en tableau numpy
img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))  # Créer la collection d'images (un seul échantillon)
img = preprocess_input(img)  # Prétraiter l'image comme le veut VGG-16

In [ ]:
y = model.predict(img)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)

In [ ]:
from keras.applications.vgg16 import decode_predictions

# Afficher les 3 classes les plus probables
print('Top 3 :', decode_predictions(y, top=3)[0])

## Transfert Learning

In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras import Model
# Charger VGG-16 pré-entraîné sur ImageNet et sans les couches fully-connected
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 3s 0us/step


In [4]:
VGG16(weights="imagenet", input_shape=(224, 224, 3)).summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
import glob2
train_folders = glob2(train_path+'/*')

In [ ]:
# Récupérer la sortie de ce réseau et ajout d'une couche de Flatten 
x = Flatten()(model.output)

# Ajouter la nouvelle couche fully-connected pour la classification à 7 classes
predictions = Dense(len(folders), activation='softmax')(x)

# Définir le nouveau modèle
new_model = Model(inputs=model.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
   layer.trainable = True

In [ ]:
for layer in model.layers:
   layer.trainable = False

In [ ]:
# Ne pas entraîner les 5 premières couches (les plus basses) 
for layer in model.layers[:5]:
   layer.trainable = False

In [ ]:
# Compiler le modèle 
new_model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

# Entraîner sur les données d'entraînement (X_train, y_train)
model_info = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2)
